In [47]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [48]:
def get_notes(files):
    notes = []

    for file in files:
        midi = converter.parse(file)

        notes_to_parse = None

        try: 
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: 
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [49]:
all_files = glob.glob("Undertale_songs/*.mid")
train_files = list(set(glob.glob("Undertale_songs/*.mid")) - set(glob.glob("Undertale_songs/087 - Hopes and Dreams.mid")))
notes = get_notes(all_files)

In [50]:
print(len(notes))

26597


In [51]:
n_vocab = len(set(notes))
n_vocab

287

In [52]:
pitchnames = sorted(set(item for item in notes))  

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [53]:
def prepare_sequences(notes, n_vocab, note_to_int):
    sequence_length = 500
    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[note] for note in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

    normalized_input = normalized_input / float(n_vocab)
    network_output = to_categorical(network_output)
    return (normalized_input, network_output)

In [54]:
train_notes = notes = get_notes(train_files)
network_input, network_output = prepare_sequences(train_notes, n_vocab, note_to_int)

In [55]:
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(256))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [59]:
def train(model, network_input, network_output):
    filepath = "model_weights/model3-{epoch:02d}.keras"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=100, batch_size=128, callbacks=callbacks_list)

In [57]:
def train_network():
    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [60]:
train_network()

Epoch 1/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - loss: 5.6307
Epoch 2/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - loss: 4.9388
Epoch 3/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - loss: 4.7536
Epoch 4/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - loss: 4.6970
Epoch 5/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - loss: 4.6163
Epoch 6/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 313s 2s/step - loss: 4.5688
Epoch 7/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - loss: 4.4841
Epoch 8/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - loss: 4.4055
Epoch 9/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - loss: 4.3232
Epoch 10/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - loss: 4.1719
Epoch 11/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 319s 2s/step - loss: 4.0337
Epoch 12/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - loss: 3.9188
Epoch 13/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - loss: 3.7797
Epoch 14/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - loss: 3.6442
Epoch 15/100
197/197 ━━━━━━━━

In [61]:
test_notes = get_notes(['Undertale_songs/087 - Hopes and Dreams.mid'])

In [62]:
network_input_test, network_output_test = prepare_sequences(test_notes, n_vocab, note_to_int)

In [63]:
def generate_notes(model, network_input, pitchnames, n_vocab, start, end):
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [64]:
model = create_network(network_input_test, n_vocab)

In [70]:
model.load_weights('model_weights/model3-55.keras')

In [71]:
def create_midi(model, network_input, pitchnames, n_vocab, start, end):
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, start, end)
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_outputs/test_output4.mid')

In [73]:
end = len(network_input_test)
#start = np.random.randint(0, end)
start = 0
create_midi(model, network_input_test, pitchnames, n_vocab, start, end)

KeyboardInterrupt: 